In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('applications data.csv')

In [7]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [8]:
df['zip5']

0          2765
1         57169
2         56721
3         35286
4          3173
          ...  
999995    54890
999996    74683
999997    86805
999998    49183
999999    39491
Name: zip5, Length: 1000000, dtype: int64

In [9]:
df['date'] = pd.to_datetime(df.date.astype(str))
df['zip5'] = df['zip5'].apply(lambda x: '{0:0>5}'.format(x))
df.loc[df.ssn==999999999,'ssn'] = -df[df.ssn==999999999]['record']
df['ssn'] = df['ssn'].apply(lambda x: '{0:0>9}'.format(x))
df.loc[df.address=='123 MAIN ST','address']= \
    df.loc[df.address=='123 MAIN ST','record'].apply(lambda x: str(x)+'RECORD')
df.loc[df.dob==19070626,'dob'] = -df.loc[df.dob==19070626]['record']
df['dob']=df['dob'].apply(lambda x: '{0:0>8}'.format(x))
df.loc[df.homephone==9999999999,'homephone']= -df.loc[df.homephone==9999999999]['record']
df['homephone'] = df['homephone'].apply(lambda x: '{0:0>10}'.format(x))

In [10]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0


### create entities

In [11]:
df['name'] = df.firstname + df.lastname
df['fulladdress'] = df.address + df.zip5
df['name_dob'] = df.name + df.dob
df['name_fulladdress'] = df.name + df.fulladdress
df['name_homephone'] = df.name + df.homephone
df['fulladdress_dob'] = df.fulladdress + df.dob
df['fulladdress_homephone'] = df.fulladdress + df.homephone
df['dob_homephone'] = df.dob + df.homephone
df['homephone_name_dob'] = df.homephone + df.name + df.dob

In [12]:
for field in list(df.iloc[:,np.r_[3:9, 12:15]].columns):
    df['ssn_' + field] = df.ssn + df[field]

In [13]:
df.columns

Index(['record', 'date', 'ssn', 'firstname', 'lastname', 'address', 'zip5',
       'dob', 'homephone', 'fraud_label', 'name', 'fulladdress', 'name_dob',
       'name_fulladdress', 'name_homephone', 'fulladdress_dob',
       'fulladdress_homephone', 'dob_homephone', 'homephone_name_dob',
       'ssn_firstname', 'ssn_lastname', 'ssn_address', 'ssn_zip5', 'ssn_dob',
       'ssn_homephone', 'ssn_name_dob', 'ssn_name_fulladdress',
       'ssn_name_homephone'],
      dtype='object')

In [14]:
attributes = list(df.iloc[:, np.r_[2,5,7,8,12:28]].columns)
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name_dob',
 'name_fulladdress',
 'name_homephone',
 'fulladdress_dob',
 'fulladdress_homephone',
 'dob_homephone',
 'homephone_name_dob',
 'ssn_firstname',
 'ssn_lastname',
 'ssn_address',
 'ssn_zip5',
 'ssn_dob',
 'ssn_homephone',
 'ssn_name_dob',
 'ssn_name_fulladdress',
 'ssn_name_homephone']

### risk table for day of week

In [31]:
import calendar
df['dow'] = df.date.apply(lambda x: calendar.day_name[x.weekday()])
train_test = df[df.date < '2016-11-01']

In [32]:
c = 4; nmid=20; y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

In [33]:
train_test.groupby('dow')['fraud_label'].mean()

dow
Friday       1e-02
Monday       1e-02
Saturday     1e-02
Sunday       1e-02
Thursday     1e-02
Tuesday      1e-02
Wednesday    2e-02
Name: fraud_label, dtype: float64

### velocity + day since

In [11]:
df1 = df.copy()
final = df.copy()
df1['check_date'] =  df1.date
df1['check_record'] = df1.record

In [12]:
for entity in attributes:
    df_1 = df1[['record','date',entity]]
    df_r = df1[['check_record','check_date',entity]]
    temp = pd.merge(df_1,df_r,left_on=entity,right_on=entity)
    
    #day since
    day_since_df = temp[temp.record > temp.check_record][['record','date','check_date']] \
            .groupby('record')[['date','check_date']].last()
    mapper = (day_since_df.date - day_since_df.check_date).dt.days
    final[entity + '_day_since'] =  final.record.map(mapper)
    final[entity + '_day_since'].fillna(365, inplace=True)
    print(f'\n{entity}_day_since ---> Done')
    
    #velocity
    for offset_t in [0,1,3,7,14,30]:
        count_day_df = temp[(temp.check_date >= (temp.Date - dt.timedelta(offset_t)))
                           & (temp.Recnum >= temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('record')[entity].count()
        final[col_name] = final.record.map(mapper2)


ssn_day_since ---> Done
ssn_count_0 ---> Done
ssn_count_1 ---> Done
ssn_count_3 ---> Done
ssn_count_7 ---> Done
ssn_count_14 ---> Done
ssn_count_30 ---> Done

address_day_since ---> Done
address_count_0 ---> Done
address_count_1 ---> Done
address_count_3 ---> Done
address_count_7 ---> Done
address_count_14 ---> Done
address_count_30 ---> Done

dob_day_since ---> Done
dob_count_0 ---> Done
dob_count_1 ---> Done
dob_count_3 ---> Done
dob_count_7 ---> Done
dob_count_14 ---> Done
dob_count_30 ---> Done

homephone_day_since ---> Done
homephone_count_0 ---> Done
homephone_count_1 ---> Done
homephone_count_3 ---> Done
homephone_count_7 ---> Done
homephone_count_14 ---> Done
homephone_count_30 ---> Done

name_dob_day_since ---> Done
name_dob_count_0 ---> Done
name_dob_count_1 ---> Done
name_dob_count_3 ---> Done
name_dob_count_7 ---> Done
name_dob_count_14 ---> Done
name_dob_count_30 ---> Done

name_fulladdress_day_since ---> Done
name_fulladdress_count_0 ---> Done
name_fulladdress_count_1 --

### relative velocity

In [13]:
start = timer()
for att in attributes:
    for d in ['0','1']:
        for dd in ['3','7','14','30']:
            final[att + '_count_' + d + '_by_' + dd] \
                = final[att + '_count_' + d] / (final[att + '_count_' + dd] / float(dd))
#print(f'Total run time: {timer() - start: 0.2f}s')

In [14]:
stats = final.describe()
stats.to_csv('project 2 variables.csv')

In [ ]:
stats = final
stats.to_csv('all project 2 variables.csv')

In [31]:
stats = final()
stats.head()

,record,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,ssn_count_30,...,ssn_name_fulladdress_count_1_by_14,ssn_name_fulladdress_count_1_by_30,ssn_name_homephone_count_0_by_3,ssn_name_homephone_count_0_by_7,ssn_name_homephone_count_0_by_14,ssn_name_homephone_count_0_by_30,ssn_name_homephone_count_1_by_3,ssn_name_homephone_count_1_by_7,ssn_name_homephone_count_1_by_14,ssn_name_homephone_count_1_by_30
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,...,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,500000.500000,0.014393,0.014410,327.100964,1.007305,1.014924,1.020140,1.026348,1.034591,1.050823,...,13.915411,29.614018,2.995813,6.977759,13.913009,29.619128,2.997168,6.980896,13.919233,29.632261
std,288675.278933,0.119104,0.000624,96.645429,0.223356,0.381210,0.423193,0.453557,0.477649,0.513331,...,0.768699,2.398058,0.079255,0.279161,0.780213,2.384740,0.065154,0.258588,0.751545,2.343096
min,1.000000,0.000000,0.013480,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.750000,2.727273,0.750000,1.166667,1.750000,2.727273,0.750000,1.166667,1.750000,2.727273
25%,250000.750000,0.000000,0.013674,365.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,14.000000,30.000000,3.000000,7.000000,14.000000,30.000000,3.000000,7.000000,14.000000,30.000000


In [32]:
final.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,...,ssn_name_fulladdress_count_1_by_14,ssn_name_fulladdress_count_1_by_30,ssn_name_homephone_count_0_by_3,ssn_name_homephone_count_0_by_7,ssn_name_homephone_count_0_by_14,ssn_name_homephone_count_0_by_30,ssn_name_homephone_count_1_by_3,ssn_name_homephone_count_1_by_7,ssn_name_homephone_count_1_by_14,ssn_name_homephone_count_1_by_30
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,000000-1,1797504115,0,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,000000-3,0216537580,0,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,0132144161,0,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0
4,5,2016-01-01,024065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,...,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0


In [33]:
stats = final.describe()
stats.drop(['count','25%','50%','75%']).to_csv('project 2 variables.csv')

In [2]:
stats = pd.read_csv('project 2 variables.csv')

In [3]:
stats.head()

,Unnamed: 0,record,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,...,ssn_name_fulladdress_count_1_by_14,ssn_name_fulladdress_count_1_by_30,ssn_name_homephone_count_0_by_3,ssn_name_homephone_count_0_by_7,ssn_name_homephone_count_0_by_14,ssn_name_homephone_count_0_by_30,ssn_name_homephone_count_1_by_3,ssn_name_homephone_count_1_by_7,ssn_name_homephone_count_1_by_14,ssn_name_homephone_count_1_by_30
0,mean,500000.500000,0.014393,0.014410,327.100964,1.007305,1.014924,1.020140,1.026348,1.034591,...,13.915411,29.614018,2.995813,6.977759,13.913009,29.619128,2.997168,6.980896,13.919233,29.632261
1,std,288675.278933,0.119104,0.000624,96.645429,0.223356,0.381210,0.423193,0.453557,0.477649,...,0.768699,2.398058,0.079255,0.279161,0.780213,2.384740,0.065154,0.258588,0.751545,2.343096
2,min,1.000000,0.000000,0.013480,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.750000,2.727273,0.750000,1.166667,1.750000,2.727273,0.750000,1.166667,1.750000,2.727273
3,max,1000000.000000,1.000000,0.015169,365.000000,21.000000,34.000000,34.000000,34.000000,34.000000,...,14.000000,30.000000,3.000000,7.000000,14.000000,30.000000,3.000000,7.000000,14.000000,30.000000


In [2]:
import defaulttimer as timer

ModuleNotFoundError: No module named 'defaulttimer'

In [15]:
from timer import timer
import datetime as dt
start = timer()
for entity in attributes:
    st = timer()
    df_1 = df1[['record','date',entity]]
    df_r = df1[['check_record','check_date',entity]]
    temp = pd.merge(df_1,df_r,left_on=entity,right_on=entity)
    print(temp)

         record       date        ssn  check_record check_date
0             1 2016-01-01  379070012             1 2016-01-01
1             2 2016-01-01  387482503             2 2016-01-01
2             2 2016-01-01  387482503        606381 2016-08-10
3             2 2016-01-01  387482503        743271 2016-09-29
4        606381 2016-08-10  387482503             2 2016-01-01
...         ...        ...        ...           ...        ...
1430249  999995 2016-12-31  026797401        999995 2016-12-31
1430250  999996 2016-12-31  766104848        999996 2016-12-31
1430251  999997 2016-12-31  445178058        999997 2016-12-31
1430252  999998 2016-12-31  025283410        999998 2016-12-31
1430253  999999 2016-12-31  361232148        999999 2016-12-31

[1430254 rows x 5 columns]
         record       date         address  check_record check_date
0             1 2016-01-01   6861 EUTST PL             1 2016-01-01
1             2 2016-01-01   7280 URASA PL             2 2016-01-01
2           

         record       date       dob_homephone  check_record check_date
0             1 2016-01-01  000000-11797504115             1 2016-01-01
1             2 2016-01-01  193406154164239415             2 2016-01-01
2             2 2016-01-01  193406154164239415        606381 2016-08-10
3             2 2016-01-01  193406154164239415        743271 2016-09-29
4        606381 2016-08-10  193406154164239415             2 2016-01-01
...         ...        ...                 ...           ...        ...
1325315  999995 2016-12-31  193603136114170353        999995 2016-12-31
1325316  999996 2016-12-31  195504188236558223        999996 2016-12-31
1325317  999997 2016-12-31  191506246790134432        999997 2016-12-31
1325318  999998 2016-12-31  198806283997657622        999998 2016-12-31
1325319  999999 2016-12-31  201203025935293731        999999 2016-12-31

[1325320 rows x 5 columns]
         record       date                   homephone_name_dob  check_record  \
0             1 2016-01-01 

KeyboardInterrupt: 